# Week 4 (Part 2): Document Similarity

In some applications, it may be difficult to define the classes that we want to use in classification ahead of time.  Or, classes might be made up various subclasses (which differ in terms of the vocabulary used).  In both of these cases (and others), it might be more appropriate to think about **document similarity**.  For a new document, can we find the most similar document in our collection?

### Preliminaries

In [2]:
import math
import sys
from nltk import word_tokenize
from Week4Labs.utils import filter_stopwords, stem, normalise
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'/Users/juliewe/resources')

from utils import * #utils has functions that we have worked on in previous weeks.

Now lets get a document collection.  Actually, lets get two collections and store them in a dictionary for easy access.

In [3]:
from sussex_nltk.corpus_readers import ReutersCorpusReader

rcr = ReutersCorpusReader()    #Create a new reader

collectionsize=10
collections={"finance":[],"sport":[]}

for key in collections.keys():
    generator=rcr.category(key).raw()
    while len(collections[key])<collectionsize:
        collections[key].append(next(generator))
        


Sussex NLTK root directory is \\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources


We now need a function which will tokenise documents and construct a *bag-of-words* document representation.  Combining some of the functionality we have been working over the past few weeks (which we have imported from utils.py), we could use something like this

In [4]:
def make_bow(somestring):
    rep=word_tokenize(somestring)
    rep=normalise(rep)
    rep=stem(rep)
    rep=filter_stopwords(rep)
    dict_rep={}
    for token in rep:
        dict_rep[token]=dict_rep.get(token,0)+1
    return(dict_rep)

In [5]:
for key, collection in collections.items():
    for doc in collection:
        print(make_bow(doc))

{'navistar': 16, 'close': 4, 'foundri': 12, 'take': 2, 'charg': 2, 'intern': 2, 'indianapoli': 8, 'cast': 2, 'employ': 1, 'num': 10, 'peopl': 1, 'late': 2, 'local': 3, 'union': 6, 'member': 3, 'reject': 2, 'compani': 8, 'propos': 4, 'truck': 4, 'school': 2, 'bu': 1, 'manufactur': 1, 'said': 8, 'tuesday': 1, 'move': 1, 'follow': 1, 'ratif': 1, 'extens': 1, 'master': 1, 'contract': 2, 'percent': 2, 'vote': 1, 'unit': 1, 'auto': 1, 'worker': 4, 'uaw': 5, 'announc': 2, 'sunday': 1, 'left': 1, 'open': 3, 'possibl': 2, 'could': 3, 'remain': 1, 'seek': 2, 'agreement': 5, 'came': 1, 'back': 2, 'say': 3, 'like': 1, 'talk': 2, 'one': 1, 'time': 2, 'alway': 1, 'work': 4, 'toward': 1, 'chairman': 1, 'john': 1, 'horn': 1, 'told': 1, 'report': 1, 'lock': 1, 'door': 2, 'jack': 1, 'laskowski': 2, 'vice': 1, 'presid': 1, 'ad': 2, 'confer': 1, 'call': 1, 'held': 1, 'jointli': 1, 'think': 1, 'know': 1, 'long': 1, 'sure': 1, 'short': 1, 'period': 1, 'offici': 4, 'gave': 1, 'detail': 1, 'except': 1, 'basic

We can apply `make_bow()` to all of the documents in our collections

In [6]:
bow_collections={key:[make_bow(doc) for doc in collection] for key,collection in collections.items()}

In [7]:
bow_collections["finance"]

[{'navistar': 16,
  'close': 4,
  'foundri': 12,
  'take': 2,
  'charg': 2,
  'intern': 2,
  'indianapoli': 8,
  'cast': 2,
  'employ': 1,
  'num': 10,
  'peopl': 1,
  'late': 2,
  'local': 3,
  'union': 6,
  'member': 3,
  'reject': 2,
  'compani': 8,
  'propos': 4,
  'truck': 4,
  'school': 2,
  'bu': 1,
  'manufactur': 1,
  'said': 8,
  'tuesday': 1,
  'move': 1,
  'follow': 1,
  'ratif': 1,
  'extens': 1,
  'master': 1,
  'contract': 2,
  'percent': 2,
  'vote': 1,
  'unit': 1,
  'auto': 1,
  'worker': 4,
  'uaw': 5,
  'announc': 2,
  'sunday': 1,
  'left': 1,
  'open': 3,
  'possibl': 2,
  'could': 3,
  'remain': 1,
  'seek': 2,
  'agreement': 5,
  'came': 1,
  'back': 2,
  'say': 3,
  'like': 1,
  'talk': 2,
  'one': 1,
  'time': 2,
  'alway': 1,
  'work': 4,
  'toward': 1,
  'chairman': 1,
  'john': 1,
  'horn': 1,
  'told': 1,
  'report': 1,
  'lock': 1,
  'door': 2,
  'jack': 1,
  'laskowski': 2,
  'vice': 1,
  'presid': 1,
  'ad': 2,
  'confer': 1,
  'call': 1,
  'held': 1,
 

## Measuring Similarity
We are going to use the cosine measure to determine how similar two documents are.  This can be defined in terms of the dot products of vectors:

\begin{eqnarray*}
\mbox{sim}_{\mbox{cosine}}(A,B) = \frac{A.B}{\sqrt{A.A \times B.B}}
\end{eqnarray*}

where the dot product of two vectors, A and B, is defined as:

\begin{eqnarray*}
A.B = \sum_{\mbox{f}} \mbox{weight}(A,f)\times \mbox{weight}(B,f) 
\end{eqnarray*}

and $\mbox{weight}(X,f)$ tells us the value associated with feature $f$ in the vector representation of $X$

### Exercise 1.1
* Write a function `dot` which takes two documents (represented as dictionaries) and returns their dot product
* Test it out on the first two documents in the finance collection.  You should get the answer 206 

In [8]:
def dot(doc1,doc2):
    the_sum=0
    for (key,value) in doc1.items():
        the_sum+=value*doc2.get(key,0)
    return the_sum


In [9]:
dot(bow_collections["finance"][0],bow_collections["finance"][1])

206

### Exercise 1.2
* Write a function `cos_sim` which takes two documents (represented as dictionaries) and returns their cosine similarity.
* Your function should make 3 calls to the `dot` function you have already defined
* If you test it out on the first two documents in the finance collection you should get 0.24 (to 2S.F.)

In [10]:
def cos_sim(dict1,dict2):
    cosine_sim=dot(dict1,dict2)/math.sqrt(dot(dict1,dict1)*dot(dict2,dict2))
    return cosine_sim

In [11]:
cos_sim(bow_collections["finance"][0],bow_collections["finance"][1])

0.2389942359865197

### Exercise 1.3
* Write some code that will compute the similarity of every document in a collection with every document in another collection
* Write code to compute the average similarity of two collections
* Compute (and display) the average similarity of 
    * the finance collection to the finance collection
    * the sport collection to the sport collection
    * the finance collection to the sport collection
    * the sport collection to the finance collection
    

## Beyond Frequency
Frequency of a word in a document does not make a very good weight because some words occur very frequently in all documents.  If two rare words occur in both of our pair of documents, that should add more to their perceived similarity than if two common words occur in both of our pair of documents.

### TF-IDF
A commonly used weight is tf-idf which stands for **term frequency, inverse document frequency**

\begin{eqnarray*}
\mbox{tf-idf}(D_i,f) = tf(D_i,f) \times idf(D_i,f)
\end{eqnarray*}

where $tf(D_i,f)$ is simply the frequency of feature f in document $D_i$
and

\begin{eqnarray*}
idf(D_i,f) = log \frac{N}{df(f)}
\end{eqnarray*}

where $N$ is the total number of documents and $\mbox{df}(f)$ is the number of documents containing $f$:  

\begin{eqnarray*}
df(f)=|\{i|\mbox{freq}(D_i,f)>0\}|
\end{eqnarray*}

The code below will take a list of documents (represented as dictionaries) and compute the document frequency for each feature.

In [12]:
def doc_freq(doclist):
    df={}
    for doc in doclist:
        for feat in doc.keys():
            df[feat]=df.get(feat,0)+1
    return df

In [13]:
#doc_freq(bow_collections['finance'])
#idf(bow_collections['finance'])

### Exercise 2.1
* Write a function which will compute the idf values for features given a list of documents
* Use it to compute idf values for features given:
    * the finance collection of documents
    * the sports collection of documents
    * the combination of the finance and sports collections
    

In [14]:
def idf(doclist):
    N=len(doclist)
    return {feat:math.log(N/v) for feat,v in doc_freq(doclist).items()}

def my_idf(doclist):
    N=len(doclist)
    df_f=0
    idf={}
    for key in doc_freq(doclist).keys():
        for i in doclist:
            for feat,v in i.items():
            
                if feat==key:
                    df_f+=1
                    break
        idf[key]=df_f
    return idf

In [ ]:
doclist={}

In [15]:
finance_idf1=my_idf(bow_collections['finance'])

In [16]:
finance_idf=idf(bow_collections['finance'])

In [17]:
test1=0
test2=0
for i,j in finance_idf.items():
    test1+=j
for i1,j1 in finance_idf.items():
    test2+=j1
print(test1,test2)

1779.8074901516707 1779.8074901516707


### Exercise 2.2
* Write a function `convert_to_tfidf` that takes two arguments:
    * a list of documents (represented as dictionaries {feat:freq})
    * a dictionary containing idf values
* and outputs a list of documents with tfidf weights (i.e., dictionaries {feat:tfidf})

In [21]:
def convert_to_tfidf(doclist,idf_dict):
    tfidf_weights_doclist={}
    for doc in doclist:
        for feat in doc.keys():
            tfidf_weights_doclist[feat]=tfidf_weights_doclist.get(feat,0)+1
    for feat,v1 in tfidf_weights_doclist.items():
        tfidf_weights_doclist[feat]=v1*idf_dict[feat]
    return tfidf_weights_doclist

In [22]:
convert_to_tfidf(bow_collections['finance'],finance_idf)

{'navistar': 2.302585092994046,
 'close': 2.302585092994046,
 'foundri': 2.302585092994046,
 'take': 2.302585092994046,
 'charg': 2.302585092994046,
 'intern': 3.6119184129778086,
 'indianapoli': 2.302585092994046,
 'cast': 2.302585092994046,
 'employ': 2.302585092994046,
 'num': 0.9482446409204371,
 'peopl': 3.2188758248682006,
 'late': 2.302585092994046,
 'local': 2.302585092994046,
 'union': 3.4657359027997265,
 'member': 3.6119184129778086,
 'reject': 3.2188758248682006,
 'compani': 3.6651629274966204,
 'propos': 3.4657359027997265,
 'truck': 2.302585092994046,
 'school': 3.2188758248682006,
 'bu': 2.302585092994046,
 'manufactur': 3.2188758248682006,
 'said': 1.7851484105136781,
 'tuesday': 3.6119184129778086,
 'move': 3.2188758248682006,
 'follow': 3.6651629274966204,
 'ratif': 2.302585092994046,
 'extens': 2.302585092994046,
 'master': 2.302585092994046,
 'contract': 2.302585092994046,
 'percent': 3.4657359027997265,
 'vote': 3.6651629274966204,
 'unit': 3.6651629274966204,
 'au

### Exercise 2.3
* Convert both of your document collections so that the weights are tfidf values rather than frequencies.  Which idf_values should you use in each case?
* Recompute the average similarity between each collection of documents (as in Ex 1.3).
* What do you notice?
* As an extension, try increasing the sizes of the collections.  What do you notice now?

In [23]:
cos_sim(convert_to_tfidf(bow_collections['finance'],finance_idf),convert_to_tfidf(bow_collections['finance'],finance_idf))

1.0